In [ ]:
! rm -rf soar && git clone https://github.com/avidcoder123/soar.git

In [ ]:
%pip install equinox openmdao[all]

In [ ]:
import sys
sys.path.append("soar")
import equinox as eqx
from soar.util import generate_base_model, density, dyn_viscousity
import jax
import jax.numpy as jnp

In [ ]:
base_model = generate_base_model()

lift_model = eqx.tree_deserialise_leaves(f"/kaggle/input/soar-surrogates/lift_surrogate.eqx", base_model)
drag_model = eqx.tree_deserialise_leaves(f"/kaggle/input/soar-surrogates/drag_surrogate.eqx", base_model)

# Variable bounds

$L\in(10^4N, 4.5\times10^6N)$

$V_\infty\in(50\frac{m}{s}, 200\frac{m}{s}$)

$h\in(10^3m, 10^4m)$

$\alpha_{geo}\in(0^\circ, 7.5^\circ)$

Note to self: in the CSV, put just the raw data of all the attempts then I can use `pandas` elsewhere to do the data analysis. This is because some conditions (like very low lift at high angle of attack and high velocity) are impossible to achieve with the optimizer bounds. Convergence can be checked using lift error.

In [ ]:
from scipy.stats import qmc

def deg2rad(rad):
    return jnp.deg2rad(rad).item()

sampler = qmc.LatinHypercube(d=3)

l_bounds = [50, 1e3, deg2rad(0)]
u_bounds = [200, 1e4, deg2rad(7.5)]

sample = sampler.random(n=10)
sample = qmc.scale(sample, l_bounds, u_bounds)
conditions = jnp.array(sample) #Test 10 different flight conditions

#0.25, 0.5, 0.75, or 1
safety_factor = 1

lifts = jnp.linspace(5e3, 1e6, 10) #10 different lift goals

In [ ]:
from soar.optimizer import Optimizer

In [ ]:
import time
from functools import partial

def run_optimization(lift_goal, conditions):
    opt = Optimizer()
    opt.load_surrogates(lift_model, drag_model)
    
    
    #Aluminum, because most wings are made using it or a composite including it
    opt.define_material(
        E=69e9,
        rho=2700,
        yield_strength=241e6,
        shear_strength=207e6
    )
    
    v_infty = conditions[0]
    altitude = conditions[1]
    alpha_geo = conditions[2]
    
    results = opt.solve_wing(
        lift_goal=lift_goal,
        v_infty=v_infty,
        rho=density(altitude),
        mu=dyn_viscousity(altitude),
        alpha_geo=alpha_geo,
        safety_factor=safety_factor
    )
    
    parameters = jnp.array(list(results["parameters"].values()))
    aerodynamics = jnp.hstack(list(results["aerodynamics"].values()))
    
    return jnp.concat([*parameters, aerodynamics])

results = []

for lift_goal in lifts:
    for condition in conditions:
        result = [lift_goal, *condition]
        t = time.time()
        result += list(run_optimization(lift_goal, condition).copy())
        result.append(time.time() - t)
        
        results.append(result)

In [ ]:
! rm -rf /kaggle/working/problem*

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data=results, columns=["lift_goal", "v_infty", "altitude", "alpha_geo", "B", "T", "P", "C", "E", "R", "b", "c", "AR", "L", "D", "Cl_0", "alpha_0", "time"])
#Convert angles to degrees
df["alpha_geo"] = jnp.rad2deg(jnp.array(np.hstack(df["alpha_geo"])))

df.to_csv(f"/kaggle/working/results_{safety_factor}.csv")